In [1]:
# Import all packages required. 
import requests
import pandas as pd
import numpy as np
from config import weather_api_key
from citipy import citipy
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations. 
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
len(coordinates)

2000

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

769

In [4]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
success_count, error_count = 0, 0

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
        # Count
        success_count += 1
        
# If an error is experienced, skip the city.
    except:
        error_count += 1
        pass
    
    # Track
    print("Retrieving " + str(i) + " of " + str(len(cities)) + "   ", end="\r")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("Success: " + str(success_count))
print("City not found: " + str(error_count))
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
-----------------------------
Data Retrieval Complete      
Success: 720
City not found: 49
-----------------------------


In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,46.06,71,75,34.52,light intensity drizzle
1,Kapaa,US,22.0752,-159.3190,75.52,79,75,3.00,broken clouds
2,Karratha,AU,-20.7377,116.8463,91.80,21,2,14.38,clear sky
3,Sao Joao Da Barra,BR,-21.6403,-41.0511,87.89,78,77,16.84,broken clouds
4,Poso,ID,-1.3959,120.7524,78.48,84,85,4.14,overcast clouds
5,Nortelandia,BR,-14.4547,-56.8028,76.91,88,100,5.59,overcast clouds
6,Tura,IN,25.5198,90.2201,58.01,65,3,3.78,clear sky
7,Hami,CN,42.8000,93.4500,24.58,43,0,6.44,clear sky
8,Fortuna,US,40.5982,-124.1573,48.65,76,90,1.01,overcast clouds
9,Xinzhi,CN,36.4989,111.7047,34.93,39,0,1.41,clear sky


In [6]:
city_data_df.to_csv("WeatherPy_Database.csv")